# Atividade 2 - MO824
RA: 171036 - F. D. DOMINGUES, Instituto de Computação (IC)

RA: 185961 - R. V. SARAIVA, Instituto de Computação (IC)

RA: 211846 - J. S. de CASTRO, Instituto de Computação (IC)

In [1]:
import os
import warnings
import json
import itertools

import numpy as np
import gurobi as gp

from gurobipy import GRB, tuplelist, quicksum
from glob import glob

In [2]:
# Funções para ler as instâncias do problema previamente geradas
def process_dist(num_vertices, dist):
    res = np.zeros((num_vertices, num_vertices))
    for k, v in dist.items():
        i,j = eval(k)
        res[i,j] = res[j,i] = v
    return res

def load_instance(filename):
    json_ins = json.load(open(filename))
    return [
        json_ins["num_vertices"],
        json_ins["points"],
        process_dist(json_ins["num_vertices"], json_ins["dist"]),
    ]

In [3]:
# Função executada pelo gurobi ao avaliar um candidato, esperamos elimitar soluções compostas por subciclos
def eliminate_subtour_solution(num_vertices, model, where):
    # Candidato a solução, segundo exemplo do Gurobi
    if where == GRB.Callback.MIPSOL:
        
        # Validando os caminhos percorridos por cada variavel
        for v in model._edges_variables:
            values = model.cbGetSolution(v)
            selected = tuplelist((i,j) for i in range(num_vertices)
                                       for j in range(num_vertices)
                                       if values[i,j] > 0.5)
            
            # Valiando subtours percorridos
            for tour in generate_subtours(num_vertices, selected):
                # Verificando se o candidato respeita as condições
                if len(tour) < num_vertices:
                    # Adiciona constraint para impedir a utilização desse subtour candidato
                    model.cbLazy(quicksum(v[i,j] for i,j in itertools.combinations(tour, 2)) <= len(tour)-1)

                
# Encontra todos os ciclos percorridos pelos vertices:
def generate_subtours(num_vertices, edges):
    unvisited = list(range(num_vertices))
    while unvisited:
        tour = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            tour.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current,'*') if j in unvisited]
        yield tour
        
        
# Função executada para gerar lista com a sequencia das cidades visitadas
def get_optimal_tour(model, num_vertices):
    tours, edges = [], []
    for var in model._edges_variables:
        # Obtendo arestas visitadas
        values = model.getAttr("x", var)
        selected = gp.tuplelist((i, j) 
                                for i in range(num_vertices)
                                for j in range(num_vertices)
                                if values[i, j] > 0.5)
        edges.append(list(selected))
        
        # Obtendo caminho percorrido (nota: apenas um subtour será gerado)
        for tour in generate_subtours(num_vertices, selected):
            tours.append(tour)

    return tours, edges

# TSP

In [4]:
instance_dir = 'Projeto 2/data'
for instance in glob(os.path.join(instance_dir, "*.json")):
    num_vertices, points, dist = load_instance(instance)
    
    model = gp.Model("tsp")
    model.setParam(gp.GRB.Param.OutputFlag, 0)
    model.Params.Threads = 4
    model.Params.Seed = 42
    model.Params.TimeLimit = 30 * 60
    
    # Definindo variavel matriz simétrica
    edges = model.addVars(num_vertices, num_vertices, vtype=GRB.BINARY, name="edges")
    for i, j in edges.keys():
        edges[j, i] = edges[i, j]
    
    # Restrição 1, nenhuma entrada pode ser de um vertice para ele mesmo
    model.addConstr(gp.quicksum([edges[i,i] for i in range(num_vertices)])==0)
    
    
    # Restrição 2, em cada linha e coluna deve haver apenas 1 selecionado (dois pela matriz ser simetrica)
    model.addConstrs(
        (gp.quicksum([edges[i,j] for j in range(num_vertices)])==2) for i in range(num_vertices)
    )
    model.addConstrs(
        (gp.quicksum([edges[i,j] for i in range(num_vertices)])==2) for j in range(num_vertices)
    )
    
    # Objetivo: minimizar custo da viagem
    model.setObjective(
        gp.quicksum(edges[i, j] * dist[i, j]/2 for i in range(num_vertices) 
                                               for j in range(num_vertices)), 
        GRB.MINIMIZE
    )
    model._edges_variables = [edges]
    model.Params.lazyConstraints = 1
    model.optimize(lambda model, where: eliminate_subtour_solution(num_vertices, model, where))
    print("n_vertices: %d" % num_vertices)
    print("custo_total: %f" % model.objVal)
    print("tempo_total: %f segundos" % model.runtime)
    print(get_optimal_tour(model, num_vertices))
    print()

Using license file /home/felipe/gurobi.lic
Academic license - for non-commercial use only
n_vertices: 20
custo_total: 3.704006
tempo_total: 0.095446 segundos
([[0, 8, 13, 15, 4, 12, 2, 3, 7, 19, 1, 14, 9, 5, 11, 18, 10, 17, 6, 16]], [[(0, 8), (0, 16), (1, 14), (1, 19), (2, 3), (2, 12), (3, 2), (3, 7), (4, 12), (4, 15), (5, 9), (5, 11), (6, 16), (6, 17), (7, 3), (7, 19), (8, 0), (8, 13), (9, 5), (9, 14), (10, 17), (10, 18), (11, 5), (11, 18), (12, 2), (12, 4), (13, 8), (13, 15), (14, 1), (14, 9), (15, 4), (15, 13), (16, 0), (16, 6), (17, 6), (17, 10), (18, 10), (18, 11), (19, 1), (19, 7)]])

n_vertices: 40
custo_total: 5.202419
tempo_total: 0.176380 segundos
([[0, 19, 9, 27, 25, 26, 23, 35, 2, 3, 37, 15, 16, 12, 13, 5, 7, 22, 36, 10, 28, 38, 33, 17, 11, 6, 18, 32, 29, 4, 14, 1, 31, 24, 39, 20, 34, 8, 30, 21]], [[(0, 19), (0, 21), (1, 14), (1, 31), (2, 3), (2, 35), (3, 2), (3, 37), (4, 14), (4, 29), (5, 7), (5, 13), (6, 11), (6, 18), (7, 5), (7, 22), (8, 30), (8, 34), (9, 19), (9, 27), (

# 2TSP

In [5]:
instance_dir = 'Projeto 2/data'
for instance in glob(os.path.join(instance_dir, "*.json")):
    num_vertices, points, dist = load_instance(instance)
    
    model = gp.Model("tsp")
    model.setParam(gp.GRB.Param.OutputFlag, 0)
    model.Params.Threads = 4
    model.Params.Seed = 42
    model.Params.TimeLimit = 30 * 60
    
    # Definindo variavel matriz simétrica
    edgesA = model.addVars(num_vertices, num_vertices, vtype=GRB.BINARY, name="edgesA")
    edgesB = model.addVars(num_vertices, num_vertices, vtype=GRB.BINARY, name="edgesB")
    for i, j in edgesA.keys():
        edgesA[j, i] = edgesA[i, j]
        edgesB[j, i] = edgesB[i, j]
    
    # Restrição 1, nenhuma entrada pode ser de um vertice para ele mesmo
    model.addConstr(gp.quicksum([edgesA[i,i] for i in range(num_vertices)])==0)
    model.addConstr(gp.quicksum([edgesB[i,i] for i in range(num_vertices)])==0)
    
    # Restrição 2, em cada linha e coluna deve haver apenas 1 selecionado (dois pela matriz ser simetrica)
    model.addConstrs(
        (gp.quicksum([edgesA[i,j] for j in range(num_vertices)])==2) for i in range(num_vertices)
    )
    model.addConstrs(
        (gp.quicksum([edgesA[i,j] for i in range(num_vertices)])==2) for j in range(num_vertices)
    )
    
    model.addConstrs(
        (gp.quicksum([edgesB[i,j] for j in range(num_vertices)])==2) for i in range(num_vertices)
    )
    model.addConstrs(
        (gp.quicksum([edgesB[i,j] for i in range(num_vertices)])==2) for j in range(num_vertices)
    )
    
    # Restrição 3, caminho de A deve ser diferente do caminho de B
    model.addConstrs(
        gp.quicksum([edgesA[i, j], edgesB[i, j]]) <= 1 for i in range(num_vertices)
                                                       for j in range(num_vertices)
    )
    
    
    # Objetivo: minimizar custo da viagem
    model.setObjective(
        gp.quicksum(edgesA[i, j] * dist[i, j]/2 for i in range(num_vertices) 
                                                for j in range(num_vertices))
        +
        gp.quicksum(edgesB[i, j] * dist[i, j]/2 for i in range(num_vertices) 
                                                for j in range(num_vertices)), 
        GRB.MINIMIZE
    )
    model._edges_variables = [edgesA, edgesB]
    model.Params.lazyConstraints = 1
    model.optimize(lambda model, where: eliminate_subtour_solution(num_vertices, model, where))
    print("n_vertices: %d" % num_vertices)
    print("custo_total: %f" % model.objVal)
    print("tempo_total: %f segundos" % model.runtime)
    print(get_optimal_tour(model, num_vertices))
    print()

n_vertices: 20
custo_total: 9.519339
tempo_total: 0.133585 segundos
([[0, 6, 3, 16, 8, 2, 12, 15, 13, 4, 14, 19, 9, 1, 7, 17, 18, 10, 5, 11], [0, 8, 13, 12, 4, 15, 2, 3, 7, 19, 1, 14, 9, 5, 18, 11, 10, 17, 6, 16]], [[(0, 6), (0, 11), (1, 7), (1, 9), (2, 8), (2, 12), (3, 6), (3, 16), (4, 13), (4, 14), (5, 10), (5, 11), (6, 0), (6, 3), (7, 1), (7, 17), (8, 2), (8, 16), (9, 1), (9, 19), (10, 5), (10, 18), (11, 0), (11, 5), (12, 2), (12, 15), (13, 4), (13, 15), (14, 4), (14, 19), (15, 12), (15, 13), (16, 3), (16, 8), (17, 7), (17, 18), (18, 10), (18, 17), (19, 9), (19, 14)], [(0, 8), (0, 16), (1, 14), (1, 19), (2, 3), (2, 15), (3, 2), (3, 7), (4, 12), (4, 15), (5, 9), (5, 18), (6, 16), (6, 17), (7, 3), (7, 19), (8, 0), (8, 13), (9, 5), (9, 14), (10, 11), (10, 17), (11, 10), (11, 18), (12, 4), (12, 13), (13, 8), (13, 12), (14, 1), (14, 9), (15, 2), (15, 4), (16, 0), (16, 6), (17, 6), (17, 10), (18, 5), (18, 11), (19, 1), (19, 7)]])

n_vertices: 40
custo_total: 13.059002
tempo_total: 2.70839